# Is it possible to just regenerate all Renditions for a post?

Would be nice to be able to regenerate all renditions for two reasons:

- Remove old renditions which are no longer needed
- Create all thumbnail / modal renditions in different formats for picture element

In [5]:
from cast import appsettings
from cast.blocks import Thumbnail


def get_all_regular_images_for_post(post):
    for image in post.images.all():
        yield image

        
def get_all_gallery_images_for_post(post):
    for gallery in post.galleries.all():
        for image in gallery.images.all():
            yield image
            

def get_all_images_for_post(post):
    for image in get_all_regular_images_for_post(post):
        yield image
    for image in get_all_gallery_images_for_post(post):
        yield image
            

def get_all_renditions(post):
    for image in get_all_images_for_post(post):
        for rendition in image.renditions.all():
            yield rendition

def get_all_required_image_renditions(post):
    image_slot_width, image_slot_height = appsettings.CAST_IMAGE_SLOT_DIMENSIONS
    
    for image in get_all_regular_images_for_post(post):
        display = Thumbnail(image, image_slot_width, image_slot_height)
        for image_format, renditions in display.renditions.items():
            for rendition in renditions:
                yield rendition


def get_all_required_gallery_renditions(post):
    image_slot_width, image_slot_height = appsettings.CAST_IMAGE_SLOT_DIMENSIONS
    thumbnail_slot_width, thumbnail_slot_height = appsettings.CAST_THUMBNAIL_SLOT_DIMENSIONS
    
    for image in get_all_gallery_images_for_post(post):
        thumbnail = Thumbnail(image, thumbnail_slot_width, thumbnail_slot_height)
        display = Thumbnail(image, image_slot_width, image_slot_height)
        for image_format, renditions in thumbnail.renditions.items():
            for rendition in renditions:
                yield rendition
        for image_format, renditions in display.renditions.items():
            for rendition in renditions:
                yield rendition                


def get_all_required_renditions(post):
    created_image_renditions = set(get_all_required_image_renditions(post))
    created_gallery_renditions = set(get_all_required_gallery_renditions(post))
    return created_image_renditions.union(created_gallery_renditions)


def delete_renditions(renditions):
    for rendition in renditions:
        rendition.delete()


def clean_up_renditions(post):

    existing_renditions = set(get_all_renditions(post))
    required_renditions = get_all_required_renditions(post)
    obsolete_renditions = existing_renditions - required_renditions
    
    # delete_renditions(obsolete_renditions)
    print(obsolete_renditions)

In [8]:
post = Post.objects.get(slug="november-2023-11-13")
clean_up_renditions(post)

set()


In [9]:
existing_renditions = set(get_all_renditions(post))

In [11]:
rendition = list(existing_renditions)[0]

In [13]:
rendition.url

'https://d2b7dn9rofvhjd.cloudfront.net/images/Nora_beim_Kinderarzt.width-1110.format-avif.avif'

In [1]:
# dir(rendition)

In [15]:
rendition.filter_spec

'width-1110|format-avif'

# Ok, I have a plan!

In [1]:
from cast import appsettings as settings
from cast.renditions import Rectangle, Width, Height, RenditionFilters

def get_all_images_from_queryset(posts):
    for post in posts:
        for image_type, image in post.get_all_images():
            yield image_type, image

In [2]:
posts_queryset = Post.objects.filter(slug="november-2023-11-13").prefetch_related("images", "galleries__images")

In [3]:
%%time
posts_queryset = Post.objects.all().prefetch_related("images", "galleries__images")
image_ids_with_filter_strings = set()
for image_type, image in get_all_images_from_queryset(posts_queryset):
    rfs = RenditionFilters.from_wagtail_image_with_type(image, image_type)
    image_pk = image.pk
    for filter_string in rfs.filter_strings:
        image_ids_with_filter_strings.add((image_pk, filter_string))

CPU times: user 112 ms, sys: 7.27 ms, total: 119 ms
Wall time: 139 ms


In [4]:
len(image_ids_with_filter_strings)

21922

In [5]:
%%time
existing_renditions = set(Rendition.objects.all().values_list("image_id", "filter_spec"))
obsolete_renditions = existing_renditions - image_ids_with_filter_strings
missing_renditions = image_ids_with_filter_strings - existing_renditions

CPU times: user 14.5 ms, sys: 3.39 ms, total: 17.9 ms
Wall time: 20.2 ms


In [13]:
obsolete_by_image_id = {}
for image_id, filter_spec in obsolete_renditions:
    obsolete_by_image_id.setdefault(image_id, []).append(filter_spec)
missing_by_image_id = {}
for image_id, filter_spec in missing_renditions:
    missing_by_image_id.setdefault(image_id, []).append(filter_spec)

In [8]:
list(obsolete_by_image_id.keys())[0]

870

In [9]:
image = Image.objects.get(pk=870)

In [11]:
image.file.name

'cast_images/originals/Xn6b8sfMS8efd3vSX8ltw.jpg'

In [12]:
obsolete_by_image_id[870]

['width-300',
 'width-768',
 'width-500',
 'max-120x80',
 'width-2200',
 'width-1100']

In [14]:
missing_by_image_id[870]

['width-214|format-avif',
 'width-107',
 'width-321',
 'width-107|format-avif',
 'width-1974',
 'width-1974|format-avif',
 'width-214',
 'width-987',
 'width-321|format-avif',
 'width-987|format-avif',
 'width-2961',
 'width-2961|format-avif']

In [28]:
image = Image.objects.get(pk=1426)

In [29]:
image.file.name

'original_images/pytest_knolling.png'

In [5]:
rendition.url

'https://d2b7dn9rofvhjd.cloudfront.net/images/Katharina_liest_Carl_Nora_und_Elli_den_Gruff.max-800x600.jpg'

In [7]:
rendition.delete()

(1, {'wagtailimages.Rendition': 1})

In [10]:
rendition = Rendition.objects.get(image_id=2091, filter_spec="max-800x600")

In [11]:
rendition

<Rendition: Rendition object (30869)>

In [17]:
existing_rendition_to_id = {(image_id, filter_spec): pk for pk, image_id, filter_spec in Rendition.objects.all().values_list("pk", "image_id", "filter_spec")}

In [19]:
#set(existing_rendition_to_id.keys())

In [20]:
rendition = Rendition.objects.get(pk=30621)

In [24]:
rendition.file.name

'images/Katharina_liest_Carl_Nora_und_Elli.width-987.format-jpeg_cXjFYrW.jpg'

In [25]:
rendition.filter_spec

'width-987|format-jpeg'